# Data Sourcing

This notebook will make API call and source data.

* S&P 500
   * P/E ratio for S&P (as a proxy for investors' emotional attitude towards the market
* Indian market's version of S&P and Dow Jones
* Market indices based on varying investment philosophies in both countries.


#### Ticker Symbols

* S & P 500 - VOO (Vanguard S&P 500 ETF(Ultra low-cost, strong tracking)
* S & P 500 Growth ETF  - VOOG (Exposure to Large-Cap US Growth companies)
* JP Morgan Equity Premium Income ETF (Actively managed) - JEPI


In [2]:
import pandas as pd, requests

In [1]:
apiKey = 'RQRA0RQ6ZZAVMA26'

In [35]:
tickers = ['VOO','VOOG','JEPI']

callAPI function:

Will take in a list of ticker symbols and return a dataframe.


In [44]:
def callAPI(tickers): 
    dfs =[]
    for ticker in tickers:
        r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize=full&apikey={apiKey}")
        APIdata = r.json()
        keys = list(APIdata.keys())
        data = APIdata[keys[1]]
        temp_df = pd.DataFrame.from_dict(data, orient='index')
        temp_df['ticker']=ticker
        dfs.append(temp_df)

    all_data = pd.concat(dfs)
    return all_data

In [45]:
callAPI(tickers)

,1. open,2. high,3. low,4. close,5. volume,ticker
2025-07-28,586.0400,586.5000,584.2400,585.4400,4193101,VOO
2025-07-25,583.8100,586.0400,583.5900,585.5800,4210090,VOO
2025-07-24,583.3400,584.7500,582.7200,583.2600,3840901,VOO
2025-07-23,580.5700,582.9600,578.8700,582.8600,4257345,VOO
2025-07-22,578.2800,578.8400,575.6400,577.9500,4767703,VOO
...,...,...,...,...,...,...
2020-05-28,51.0600,56.2700,50.9200,51.1400,53336,JEPI
2020-05-27,50.7200,50.7200,50.4000,50.6270,352088,JEPI
2020-05-26,50.5000,50.6000,50.2380,50.2380,5638,JEPI
2020-05-22,49.9300,50.1000,49.8900,50.0940,9518,JEPI


In [30]:
r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=JEPI&outputsize=full&apikey={apiKey}")

In [34]:
data = r.json()
keys = list(data.keys())
data[keys[1]]

{'2025-07-28': {'1. open': '57.3500',
  '2. high': '57.3500',
  '3. low': '57.1300',
  '4. close': '57.1700',
  '5. volume': '4131079'},
 '2025-07-25': {'1. open': '57.2300',
  '2. high': '57.3250',
  '3. low': '57.1335',
  '4. close': '57.3200',
  '5. volume': '3306195'},
 '2025-07-24': {'1. open': '57.2300',
  '2. high': '57.2900',
  '3. low': '57.1500',
  '4. close': '57.1700',
  '5. volume': '3941691'},
 '2025-07-23': {'1. open': '57.1600',
  '2. high': '57.2665',
  '3. low': '57.0817',
  '4. close': '57.2300',
  '5. volume': '3629041'},
 '2025-07-22': {'1. open': '56.8600',
  '2. high': '57.1799',
  '3. low': '56.8535',
  '4. close': '57.1500',
  '5. volume': '3131760'},
 '2025-07-21': {'1. open': '57.0400',
  '2. high': '57.0400',
  '3. low': '56.8350',
  '4. close': '56.8500',
  '5. volume': '3687551'},
 '2025-07-18': {'1. open': '57.1200',
  '2. high': '57.1200',
  '3. low': '56.8800',
  '4. close': '56.9700',
  '5. volume': '7796603'},
 '2025-07-17': {'1. open': '56.6500',
  '

In [22]:
(data[keys[1]])

{'2025-07-28': {'1. open': '57.3500',
  '2. high': '57.3500',
  '3. low': '57.1300',
  '4. close': '57.1700',
  '5. volume': '4131079'},
 '2025-07-25': {'1. open': '57.2300',
  '2. high': '57.3250',
  '3. low': '57.1335',
  '4. close': '57.3200',
  '5. volume': '3306195'},
 '2025-07-24': {'1. open': '57.2300',
  '2. high': '57.2900',
  '3. low': '57.1500',
  '4. close': '57.1700',
  '5. volume': '3941691'},
 '2025-07-23': {'1. open': '57.1600',
  '2. high': '57.2665',
  '3. low': '57.0817',
  '4. close': '57.2300',
  '5. volume': '3629041'},
 '2025-07-22': {'1. open': '56.8600',
  '2. high': '57.1799',
  '3. low': '56.8535',
  '4. close': '57.1500',
  '5. volume': '3131760'},
 '2025-07-21': {'1. open': '57.0400',
  '2. high': '57.0400',
  '3. low': '56.8350',
  '4. close': '56.8500',
  '5. volume': '3687551'},
 '2025-07-18': {'1. open': '57.1200',
  '2. high': '57.1200',
  '3. low': '56.8800',
  '4. close': '56.9700',
  '5. volume': '7796603'},
 '2025-07-17': {'1. open': '56.6500',
  '

In [57]:
ETF = requests.get(f"https://www.alphavantage.co/query?function=ETF_PROFILE&symbol=SPG&outputsize=full&apikey={apiKey}")

In [58]:
ETFdata = ETF.json()

In [59]:
ETFdata

{}